<a href="https://colab.research.google.com/github/chamodi08jaya/Generative-Adversarial-Networks-GANs-Specialization/blob/main/Copy_of_Test_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

An example script for using texturegan model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My Drive/TextureGanPython3

/content/drive/My Drive/TextureGanPython3


In [ ]:
from argparser import parse_arguments

In [ ]:
#dummy command
command = '--display_port 7770 --load 0 --load_D -1 --load_epoch 105 --gpu 2 --model texturegan --feature_weight 1e2 --pixel_weight_ab 1e3 --global_pixel_weight_l 1e3 --local_pixel_weight_l 0 --style_weight 0 --discriminator_weight 1e3 --discriminator_local_weight 1e6  --learning_rate 1e-4 --learning_rate_D 1e-4 --batch_size 36 --save_every 50 --num_epoch 100000 --save_dir dotted/ --load_dir dotted/ --data_path training_handbags_pretrain/ --learning_rate_D_local  1e-4 --local_texture_size 50 --patch_size_min 20 --patch_size_max 40 --num_input_texture_patch 1 --visualize_every 5 --num_local_texture_patch 1'
args = parse_arguments(command.split())


In [ ]:
!pip install visdom

     |████████████████████████████████| 676 kB 5.3 MB/s 
     |████████████████████████████████| 52 kB 1.3 MB/s 
  Created wheel for visdom: filename=visdom-0.1.8.9-py3-none-any.whl size=655250 sha256=b88e98ab971fb1f0bf66adae6812be3b13bf4f9c2a492b35a65b3e64ab2c7f93
  Stored in directory: /root/.cache/pip/wheels/2d/d1/9b/cde923274eac9cbb6ff0d8c7c72fe30a3da9095a38fd50bbf1
  Created wheel for torchfile: filename=torchfile-0.1.0-py3-none-any.whl size=5710 sha256=48e9e8f89f9805eb725a7e2a9ef3102c553791ffa8d22c9fb11d36bc25ad6a78
  Stored in directory: /root/.cache/pip/wheels/ac/5c/3a/a80e1c65880945c71fd833408cd1e9a8cb7e2f8f37620bb75b
Successfully built visdom torchfile


In [ ]:
!pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org torch===0.4.1 torchvision===0.2.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 519.5 MB 23 kB/s 
     |████████████████████████████████| 54 kB 2.0 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Successfully uninstalled torchvision-0.10.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 0.4.1 which is incompatible.
fastai 1.0.61 requires torch>=1.0.0, but you have torch 0.4.1 which is incompatible.


In [ ]:
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb

!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub
!apt-get update
!apt-get install cuda=8.0.61-1
!apt autoremove

--2021-09-21 22:19:24--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/8.0/prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb [following]
--2021-09-21 22:19:24--  https://developer.nvidia.com/compute/cuda/8.0/prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?1MgTro6dVHtrFx3plUWGF7B8KdTYVkw8fxrhEc2FzXb96gcJOgXGHTj47Pf9UEh8EZ2JHEIIk4DIQ

In [ ]:
from main import get_transforms
from dataloader import imfol
from dataloader.imfol import ImageFolder, make_dataset
import torch
from torch.utils.data.sampler import SequentialSampler
from torch.utils.data import DataLoader
import math
from torch.autograd import Variable
from utils.visualize import vis_patch, vis_image

## Load from validation folder

In [ ]:
args.batch_size = 1
args.image_size =152
args.resize_max = 256
args.resize_min = 256
args.data_path = 'training_handbags_pretrain/' #change to your data path

In [ ]:
transform = get_transforms(args)
val = make_dataset(args.data_path, 'val')
valDset = ImageFolder('val', args.data_path, transform)
val_display_size = 1
valLoader = DataLoader(dataset=valDset, batch_size=val_display_size,shuffle=True)

## Load Pretrained model

In [ ]:
def load_network(model, save_path):
        
    model_state = torch.load(save_path)
    
    if "state_dict" in model_state:
        model.load_state_dict(model_state["state_dict"])
    else:
        model.load_state_dict(model_state)

        model_state = {
            'state_dict': model.cpu().state_dict(),
            'epoch': epoch,
            'iteration': iteration,
            'model': args.model,
            'color_space': args.color_space,
            'batch_size': args.batch_size,
            'dataset': dataset,
            'image_size': args.image_size
        }
    
    model.cuda()


In [ ]:
!wget https://s3-us-west-2.amazonaws.com/texturegan/textureD_final_allloss_handbag_3300.pth

--2021-09-21 22:23:03--  https://s3-us-west-2.amazonaws.com/texturegan/textureD_final_allloss_handbag_3300.pth
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.92.128.216
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.92.128.216|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32077381 (31M) [application/x-www-form-urlencoded]
Saving to: ‘textureD_final_allloss_handbag_3300.pth.1’

textureD_final_alll 100%[===================>]  30.59M  22.7MB/s    in 1.3s    

2021-09-21 22:23:05 (22.7 MB/s) - ‘textureD_final_allloss_handbag_3300.pth.1’ saved [32077381/32077381]



In [ ]:
from models import texturegan,discriminator

#change to your location
model_location = 'textureD_final_allloss_handbag_3300.pth'

netG = texturegan.TextureGAN(5, 3, 32)
load_network(netG, model_location)

netG.eval()

TextureGAN(
  (model): Sequential(
    (main_model): MainModel(
      (main_model): Sequential(
        (conv_1): Conv2d(5, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (batch_1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (norm_1): ReLU(inplace)
        (res_block_1): ResidualBlock(
          (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace)
          (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (conv_2): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (batch_2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (norm_2): ReLU(inplace)
        (res_block_2): Residual

In [ ]:
data = valLoader.__iter__().__next__()
from utils import transforms as custom_transforms

StopIteration: ignored

In [ ]:
def get_input(val_loader,xcenter,ycenter,patch_size,num_patch):
    img, skg, seg, eroded_seg, txt = val_loader
    img = custom_transforms.normalize_lab(img)
    skg = custom_transforms.normalize_lab(skg)
    txt = custom_transforms.normalize_lab(txt)
    seg = custom_transforms.normalize_seg(seg)
    eroded_seg = custom_transforms.normalize_seg(eroded_seg)

    bs, w, h = seg.size()

    seg = seg.view(bs, 1, w, h)
    seg = torch.cat((seg, seg, seg), 1)

    eroded_seg = eroded_seg.view(bs, 1, w, h)
    eroded_seg = torch.cat((eroded_seg, eroded_seg, eroded_seg), 1)

    temp = torch.ones(seg.size()) * (1 - seg).float()
    temp[:, 1, :, :] = 0  # torch.ones(seg[:,1,:,:].size())*(1-seg[:,1,:,:]).float()
    temp[:, 2, :, :] = 0  # torch.ones(seg[:,2,:,:].size())*(1-seg[:,2,:,:]).float()

    txt = txt.float() * seg.float() + temp

    patchsize = args.local_texture_size
    batch_size = bs
    if xcenter < 0 or ycenter < 0:
        inp, texture_loc = gen_input_rand(txt, skg, eroded_seg[:, 0, :, :] * 100,
                                              patch_size, patch_size,
                                              num_patch)
    else:
        inp, texture_loc = gen_input_exact(txt, skg, eroded_seg[:, 0, :, :] * 100,xcenter,ycenter,patch_size,1)
        
    return inp,texture_loc 
def get_inputv(inp):
    input_stack = torch.FloatTensor().cuda()
    input_stack.resize_as_(inp.float()).copy_(inp)
    inputv = Variable(input_stack)
    return inputv

In [ ]:
from train import gen_input, rand_between, gen_input_rand

In [ ]:
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import numpy as np

In [ ]:
color_space = 'lab'

data = valLoader.__iter__().__next__()

img, skg, seg, eroded_seg, txt = data

img = custom_transforms.normalize_lab(img)
skg = custom_transforms.normalize_lab(skg)
txt = custom_transforms.normalize_lab(txt)
seg = custom_transforms.normalize_seg(seg)
eroded_seg = custom_transforms.normalize_seg(eroded_seg)
inp,texture_loc = get_input(data,-1,-1,30,1)

seg = seg!=0

model = netG

inpv = get_inputv(inp.cuda())
output = model(inpv.cuda())

out_img = vis_image(custom_transforms.denormalize_lab(output.data.double().cpu()),
                                    color_space)
inp_img = vis_patch(custom_transforms.denormalize_lab(txt.cpu()),
                            custom_transforms.denormalize_lab(skg.cpu()),
                            texture_loc,
                            color_space)
tar_img = vis_image(custom_transforms.denormalize_lab(img.cpu()),
                        color_space)

plt.figure()
plt.imshow(np.transpose(inp_img[0],(1, 2, 0)))
plt.axis('off')
#plt.figure()  
plt.figure()
plt.imshow(np.transpose(out_img[0],(1, 2, 0)))
plt.axis('off')

StopIteration: ignored

In [ ]:
!wget https://s3-us-west-2.amazonaws.com/texturegan/training_handbag.tar.gz

--2021-09-21 19:09:53--  https://s3-us-west-2.amazonaws.com/texturegan/training_handbag.tar.gz
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 3.5.79.184
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|3.5.79.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3799380466 (3.5G) [application/x-gzip]
Saving to: ‘training_handbag.tar.gz’

training_handbag.ta 100%[===================>]   3.54G  37.6MB/s    in 1m 42s  

2021-09-21 19:11:36 (35.4 MB/s) - ‘training_handbag.tar.gz’ saved [3799380466/3799380466]



In [ ]:
!tar -xvf training_handbag.tar.gz

training_handbags_pretrain/
training_handbags_pretrain/train_seg/
training_handbags_pretrain/train_seg/wendy/
training_handbags_pretrain/train_seg/wendy/110702_AB.jpg
^C
